# Get the data

## Statcast data

In [16]:
import pandas as pd
import requests
import io

# pybaseball was breaking for some reason so I pulled this method out of the source code
def small_request(start_dt,end_dt):
    url = "https://baseballsavant.mlb.com/statcast_search/csv?all=true&hfPT=&hfAB=&hfBBT=&hfPR=&hfZ=&stadium=&hfBBL=&hfNewZones=&hfGT=R%7CPO%7CS%7C=&hfSea=&hfSit=&player_type=pitcher&hfOuts=&opponent=&pitcher_throws=&batter_stands=&hfSA=&game_date_gt={}&game_date_lt={}&team=&position=&hfRO=&home_road=&hfFlag=&metric_1=&hfInn=&min_pitches=0&min_results=0&group_by=name&sort_col=pitches&player_event_sort=h_launch_speed&sort_order=desc&min_abs=0&type=details&".format(start_dt, end_dt)
    s=requests.get(url, timeout=None).content
    data = pd.read_csv(io.StringIO(s.decode('utf-8')))#, error_bad_lines=False) # skips 'bad lines' breaking scrapes. still testing this.
    return data

date_range = pd.date_range('2018-03-29', pd.datetime.today()).strftime('%Y-%m-%d')
small_request(start_dt=date_range[0], end_dt=date_range[0])

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,CH,2018-03-29,83.5,-2.6238,5.0668,Noe Ramirez,543760,598287,single,hit_into_play_score,...,5,5,5,5,5,5,5,5,Strategic,Strategic
1,CH,2018-03-29,82.7,-2.6167,5.0671,Noe Ramirez,543760,598287,NaN,foul,...,5,5,5,5,5,5,5,5,Strategic,Strategic
2,CU,2018-03-29,77.2,-2.9329,4.8254,Noe Ramirez,543760,598287,NaN,foul,...,5,5,5,5,5,5,5,5,Strategic,Strategic
3,CU,2018-03-29,76.6,-2.8895,4.8506,Noe Ramirez,543760,598287,NaN,foul,...,5,5,5,5,5,5,5,5,Strategic,Strategic
4,CU,2018-03-29,77.8,-2.8185,4.8878,Noe Ramirez,543760,598287,NaN,ball,...,5,5,5,5,5,5,5,5,Strategic,Strategic
5,CU,2018-03-29,77.9,-2.8218,4.8757,Noe Ramirez,543760,598287,NaN,ball,...,5,5,5,5,5,5,5,5,Strategic,Strategic
6,CH,2018-03-29,84.1,-2.7118,4.9995,Noe Ramirez,621471,598287,triple,hit_into_play_no_out,...,5,5,5,5,5,5,5,5,Standard,Standard
7,SI,2018-03-29,89.3,-2.9245,4.8011,Noe Ramirez,621471,598287,NaN,foul,...,5,5,5,5,5,5,5,5,Standard,Standard
8,FC,2018-03-29,92.1,-2.6671,5.5153,Jacob Barnes,605486,606930,strikeout,swinging_strike,...,1,2,1,2,2,1,1,2,Infield shift,Standard
9,SI,2018-03-29,88.8,-2.7637,4.8414,Noe Ramirez,621471,598287,NaN,swinging_strike,...,5,5,5,5,5,5,5,5,Standard,Standard


In [21]:
from concurrent.futures import ThreadPoolExecutor, as_completed

date_range = pd.date_range('2018-03-29', pd.datetime.today()).strftime('%Y-%m-%d')
results = []
errors = []
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(small_request, date, date): date for date in date_range}
    for future in as_completed(futures):
        try:
            results.append(future.result())
        except Exception as err:
            errors.append(err)
            print(f'could not get data for data {futures[future]}')

In [22]:
df = pd.concat(results)
df.shape

/Users/dgates/venvs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


(708746, 90)

In [23]:
df.to_csv(f'statcast-{pd.datetime.today().strftime("%Y-%m-%d")}.csv', index=False)

In [24]:
d = df.groupby([
    'game_date', 'home_team', 'away_team', 
], as_index=False)[['post_away_score', 'post_home_score']].max()

In [25]:
d[(d.home_team == 'PHI') & (d.away_team == 'SF')]

,game_date,home_team,away_team,post_away_score,post_home_score
486,2018-05-07,PHI,SF,0.0,11.0
499,2018-05-08,PHI,SF,2.0,4.0
514,2018-05-09,PHI,SF,3.0,11.0
526,2018-05-10,PHI,SF,3.0,6.0


In [26]:
d.shape

(2355, 5)

In [27]:
team_ids = {team: i for i, team in enumerate(d.home_team.unique())}
n_teams = len(team_ids)
team_ids

{'ARI': 0,
 'ATL': 1,
 'BAL': 2,
 'BOS': 21,
 'CHC': 29,
 'CIN': 13,
 'CLE': 27,
 'COL': 28,
 'CWS': 22,
 'DET': 14,
 'HOU': 15,
 'KC': 3,
 'LAA': 16,
 'LAD': 4,
 'MIA': 5,
 'MIL': 17,
 'MIN': 23,
 'NYM': 6,
 'NYY': 19,
 'OAK': 7,
 'PHI': 24,
 'PIT': 18,
 'SD': 8,
 'SEA': 9,
 'SF': 20,
 'STL': 25,
 'TB': 10,
 'TEX': 11,
 'TOR': 12,
 'WSH': 26}

In [28]:
import numpy as np
d['home_team_id'] = d.home_team.map(team_ids)
d['away_team_id'] = d.away_team.map(team_ids)
d['home_team_win'] = np.where(d.post_home_score > d.post_away_score, 1, 0)
d['away_team_win'] = np.where(d.post_home_score < d.post_away_score, 1, 0)

## Standings

In [29]:
import requests as rq
import pandas as pd
response = rq.get('https://projects.fivethirtyeight.com/2018-mlb-predictions/')
standings = pd.read_html(response.content)[0]

In [30]:
standings

,Unnamed: 0_level_0,Chance of making …,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0
,Team▲▼,Division▲▼,Team rating▲▼,1-week change▲▼,Div. SeriesMake Div. Series▲▼,League champ.Make league champ.▲▼,World SeriesMake W.S.▲▼,Win World SeriesWin W.S.▲▼
0,Red Sox108-54Red Sox,AL East,1600,15.0,✓,✓,✓,61%
1,Dodgers92-71Dodgers,NL West,1582,-1.0,✓,✓,✓,39%
2,Astros103-59Astros,AL West,1598,-16.0,✓,✓,—,—
3,Brewers96-67Brewers,NL Central,1561,1.0,✓,✓,—,—
4,Yankees100-62Yankees,AL East,1565,NaN,✓,—,—,—
5,Rockies91-72Rockies,NL West,1526,NaN,✓,—,—,—
6,Indians91-71Indians,AL Central,1545,NaN,✓,—,—,—
7,Braves90-72Braves,NL East,1515,NaN,✓,—,—,—
8,Athletics97-65Athletics,AL West,1562,NaN,—,—,—,—


In [31]:
team_name_map = {
    'Diamondbacks': 'ARI',
    'Braves': 'ATL',
    'Orioles': 'BAL',
    'Red Sox': 'BOS',
    'Angels': 'LAA',
    'Cubs': 'CHC',
    'White Sox': 'CWS',
    'Reds': 'CIN',
    'Indians': 'CLE',
    'Rockies': 'COL',
    'Tigers': 'DET',
    'Astros': 'HOU',
    'Royals': 'KC',
    'Dodgers': 'LAD',
    'Marlins': 'MIA',
    'Brewers': 'MIL',
    'Twins': 'MIN',
    'Yankees': 'NYY',
    'Athletics': 'OAK',
    'Phillies': 'PHI',
    'Pirates': 'PIT',
    'Padres': 'SD',
    'Mariners': 'SEA',
    'Giants': 'SF',
    'Cardinals': 'STL',
    'Rays': 'TB',
    'Rangers': 'TEX',
    'Blue Jays': 'TOR',
    'Nationals': 'WSH',
    'Mets': 'NYM',
}

In [32]:
def sanitize_name(n):
    out = ''
    for a in n:
        if a.isdigit():
            break
        out += a
    return out
rankings = standings.iloc[:,0].apply(sanitize_name).map(team_name_map)

In [33]:
import numpy as np
rankings = pd.DataFrame({
    'team': rankings,
    'team_id': rankings.map(team_ids),
    'score': np.arange(len(rankings))[::-1]
})

In [34]:
rankings

,team,team_id,score
0,BOS,21,29
1,LAD,4,28
2,HOU,15,27
3,MIL,17,26
4,NYY,19,25
5,COL,28,24
6,CLE,27,23
7,ATL,1,22
8,OAK,7,21
9,CHC,29,20


In [35]:
assert not rankings.team.isnull().any()
assert not rankings.team_id.isnull().any()

In [36]:
# center at 10 - negative values restrict how we can model the problem
rankings['score'] = 10 + (rankings['score'] - np.mean(rankings['score'])) / np.std(rankings['score'])

In [37]:
rankings

,team,team_id,score
0,BOS,21,11.675247
1,LAD,4,11.559712
2,HOU,15,11.444178
3,MIL,17,11.328644
4,NYY,19,11.213110
5,COL,28,11.097575
6,CLE,27,10.982041
7,ATL,1,10.866507
8,OAK,7,10.750973
9,CHC,29,10.635438


In [38]:
rankings.to_csv(f'rankings-{pd.datetime.today().strftime("%Y-%m-%d")}.csv', index=False)

# divisions

In [39]:
division_map = {
    'AL Central': ['CWS', 'CLE', 'DET', 'KC', 'MIN'],
    'AL East': ['BAL', 'BOS', 'NYY', 'TB', 'TOR'],
    'AL West': ['HOU', 'LAA', 'OAK', 'SEA', 'TEX'],
    'NL Central': ['CHC', 'CIN', 'MIL', 'PIT', 'STL'],
    'NL East': ['ATL', 'MIA', 'NYM', 'PHI', 'WSH'],
    'NL West': ['ARI', 'COL', 'LAD', 'SD', 'SF'],
}
# division_map = {team: division for division, teams in division_map.items() for team in teams}
division_map

{'AL Central': ['CWS', 'CLE', 'DET', 'KC', 'MIN'],
 'AL East': ['BAL', 'BOS', 'NYY', 'TB', 'TOR'],
 'AL West': ['HOU', 'LAA', 'OAK', 'SEA', 'TEX'],
 'NL Central': ['CHC', 'CIN', 'MIL', 'PIT', 'STL'],
 'NL East': ['ATL', 'MIA', 'NYM', 'PHI', 'WSH'],
 'NL West': ['ARI', 'COL', 'LAD', 'SD', 'SF']}

In [40]:
import json
json.dump(division_map, open('divisions.json', 'w'))

## Aggregate

In [41]:
dd = d.groupby(['home_team', 'home_team_id', 'away_team', 'away_team_id'], as_index=False)[['home_team_win', 'away_team_win']].sum()
dd['total_games'] = dd.home_team_win + dd.away_team_win
dd.head()

,home_team,home_team_id,away_team,away_team_id,home_team_win,away_team_win,total_games
0,ARI,0,ATL,1,1,3,4
1,ARI,0,CHC,29,1,2,3
2,ARI,0,CIN,13,2,1,3
3,ARI,0,COL,28,3,6,9
4,ARI,0,HOU,15,0,1,1


In [42]:
dd.to_csv(f'wins-{pd.datetime.today().strftime("%Y-%m-%d")}.csv', index=False)